# Modulo 07 - Bando de Dados

In [9]:
import pandas as pd
import numpy as np
import re

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

C:\Users\pedro\AppData\Local\Temp\ipykernel_11312\2226304558.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [62]:
data = pd.read_csv('C:/Users/pedro/OneDrive/Documentos/CURSOS/Python do DS ao DEV/products_hm.csv')

# product id
data = data.dropna(subset=['product_id'])
data['product_id'] = data['product_id'].astype(int)

# product name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '')).astype(float)
#
# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style id
data['style_id'] = data['style_id'].astype(int)

# color id
data['color_id'] = data['color_id'].astype(int)

# color name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

# fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# size number
data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# size model
data['size_model'] = data['size'].str.extract('(\d+/\\d+)'
                                             )


# composition
data = data[~data['composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['composition'].str.contains('Lining:', na=False)]
data = data[~data['composition'].str.contains('Shell:', na=False)]

# drop duplicates
data = data.drop_duplicates(subset=['product_id', 'product_category', 'product_name', 'product_price',
                                    'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last')
    
# reset index
data = data.reset_index(drop=True)

# break composition by comma
df1 = data['composition'].str.split(',', expand=True)

# cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['cotton', 'polyester', 'elastane', 'elasterell'])

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df1[0]], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('polyester 0%')

# elastano
df_elastane = df1.loc[df1[1].str.contains('Elastane', na=True), 1]
df_elastane.name = 'elastane'

# combine elastane from both columns 1 and 2
df_elastane = df_elastane.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elastane'] = df_ref['elastane'].fillna('Elastane 0%')

# elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na=True), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna('Elasterell 0%')

# final join
data = pd.concat([data, df_ref], axis=1)

# format composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elastane'] = data['elastane'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)

# Drop columns
data = data.drop(columns=['size', 'product safety', 'composition'], axis=1)

# Drop duplicates
data = data.drop_duplicates()
data.shape

(308, 15)

In [64]:
data.head(5)

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,636207006,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207,6,white,slim_fit,187,31/32,0.99,0.0,0.01,0.0
1,636207006,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207,6,dark_denim_blue,slim_fit,187,31/30,0.88,0.1,0.02,0.0
2,636207006,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207,6,dark_gray_denim,slim_fit,187,31/30,0.88,0.1,0.02,0.0
3,636207006,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207,6,gray,slim_fit,187,31/30,0.88,0.1,0.02,0.0
4,636207006,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207,6,black,slim_fit,187,31/30,0.88,0.1,0.02,0.0


In [65]:
data.dtypes

product_id                   int32
product_category            object
product_name                object
product_price              float64
scrapy_datetime     datetime64[ns]
style_id                     int32
color_id                     int32
color_name                  object
fit                         object
size_number                 object
size_model                  object
cotton                     float64
polyester                  float64
elastane                   float64
elasterell                 float64
dtype: object